In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

movielens_train = pd.read_csv('data/movielens_train.csv', index_col=0,encoding='latin-1')
movielens_test = pd.read_csv('data/movielens_test.csv', index_col=0,encoding='latin-1')
users = pd.read_table('data/users.dat',
                      sep='::', header=None,
                      names=['user_id', 'gender', 'age', 'occupation', 'zip'])



/Users/leey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [2]:
print(movielens_test.columns)
print(movielens_test.shape)

Index(['user_id', 'movie_id', 'rating', 'timestamp', 'gender', 'age',
       'occupation', 'zip', 'title', 'genres', 'for_testing'],
      dtype='object')
(2668, 11)


In [3]:
def rmse(pred_y, true_y):
    """root mean squared error"""
    return np.sqrt(np.mean(np.power(pred_y - true_y, 2)))

In [4]:
def score_eval(pred_func):
    """function to evaluate the prediction function"""
    # user id and movie id
    user_movie = zip(movielens_test.user_id, movielens_test.movie_id)
    # for each user, apply the prediction function
    pred_score = np.array([pred_func(u, i) for u, i in user_movie])
    true_score = movielens_test.rating
    # return the prediction error
    return rmse(pred_score, true_score)

In [5]:
def euclidean(s1, s2):
    """calculate euclidean distance for two series"""
    return 1/(1 + np.sqrt(np.sum(np.power(s1 - s2, 2))))

In [63]:
def cosine(s1, s2):
    """calculate cosine similarity for two series"""
    return np.sum(s1 * s2) / np.sqrt(np.sum(s1 ** 2) * np.sum(s2 ** 2))

In [187]:
from collections import defaultdict

# content based filter:
# 0. convert genres into categories
# 1. keep log of user's history
# 2. search user's history to make rating

def make_user_log(movie_df):
    # dictionary with user keys and np arrays of movie id and ratings
    user_id = set(movielens_test.user_id)
    user_dict = dict.fromkeys(user_id)
    movie_genre = pd.get_dummies(movielens_test.genres).reset_index(drop=True)
    for key in user_dict.keys():
        idx = movielens_test.user_id == key
        movie_id_rating = movielens_test[idx][['movie_id', 'rating']].reset_index(drop=True)
        temp = movie_genre[idx.values.reshape(-1,1)]
        user_dict[key] = pd.concat([movie_id_rating, temp], axis=1)
    return user_dict
    
def search_user_log(user_dict, user):
    return user_dict[user]

In [192]:
# users = movielens_test['user_id'].iloc[:10].reset_index(drop=True)
# movie_rating = movielens_test[['movie_id', 'rating']].iloc[:10].reset_index(drop=True)

test = movielens_test.iloc[:10]

# user_id = set(users)
# user_dict = dict.fromkeys(user_id
movie_genre = pd.get_dummies(movielens_test.genres).reset_index(drop=True)
idx = movielens_test.user_id == 5
movie_genre[idx.values.reshape(-1,1)]
# movie_genre = pd.get_dummies(test.genres)
# make_user_log(test)

,Action,Action|Adventure,Action|Adventure|Animation|Horror|Sci-Fi,Action|Adventure|Children's|Comedy,Action|Adventure|Children's|Fantasy,Action|Adventure|Comedy,Action|Adventure|Comedy|Crime,Action|Adventure|Comedy|Horror,Action|Adventure|Comedy|Horror|Sci-Fi,Action|Adventure|Comedy|Romance,...,Mystery|Thriller,Romance,Romance|Thriller,Sci-Fi,Sci-Fi|Thriller,Sci-Fi|Thriller|War,Sci-Fi|War,Thriller,War,Western
1009,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
s = x.iloc[0, 3:]
y = x.iloc[0:5,3:].T
y.apply(lambda x: cosine(x, s), axis=0)
# cosine(x.iloc[0,3:], x.iloc[0, 3:])

0    1.0
1    0.0
2    0.0
3    0.0
4    0.0
dtype: float64

In [189]:
class ContentCustom:
    """ Content filtering. """
    def __init__(self, movie_df, curr_user):
        """ Prepare datastructures for estimation. """
        
        self.all_user_profiles = make_user_log(movie_df)
        self.curr_user_profile = search_user_log(self.all_user_profiles, curr_user)

    def euclidean(self):
        self.func = euclidean

    def cosine(self):
        self.func = cosine

    def estimate(self, pred_movie):
        """ Ratings weighted by correlation similarity. """
        
        # find the closest movie the user has rated before
        hist_movie = self.curr_user_profile.iloc[:, 3:].T
        sims = hist_movie.apply(lambda x: self.func(x, pred_movie), axis=0)
        
        ratings_sims = pd.DataFrame({'sim': sims, 'rating': self.cur_user_profile.rating})
        ratings_sims = ratings_sims[ratings_sims.sim == ratings_sims.sim.max(axis=0)]
        
        if ratings_sims.empty:
            return 2.5
        else:
            return ratings_sims.rating

In [191]:
content_filtering = ContentCustom(movielens_test, 5042)
content_filtering.cosine
content_filtering.estimate()

<bound method ContentCustom.cosine of <__main__.ContentCustom object at 0x1c7deaa90>>